In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with open('dataset/gold_label_dataset.pkl', 'rb') as file:
    df = pd.compat.pickle_compat.load(file) 

In [ ]:
df["label_M_gold_main"].apply(len)>1

In [ ]:
df[df["label_M_gold_main"].apply(len)>1]

In [ ]:
# i = 166
i = 30
print(df[df["label_M_gold_main"].apply(len)>1].iloc[i].seeds)
df[df["label_M_gold_main"].apply(len)>1].iloc[i]

In [ ]:
emotions = df.columns[df.columns.str.startswith("M_")].values.tolist()
emotions = [e[2:].lower() for e in emotions]
emotions

## Analysis on distribution of labels

In [ ]:
cols = df.columns[df.columns.str.startswith("label_")].tolist()
freq_counts = {e: dict.fromkeys(emotions, 0) for e in cols}
for index, row in df.iterrows():
    for column in cols:
        for elem in row[column]:
            freq_counts[column][elem] += 1

Some of the emotions are overrepresented, specifically "joy" and "something else". On the other hand, "fear" and "trust" are somewhat underrepresented. This is true for all types of labels, as the distribution is mostly the same across them, with some minor differences between multimodal and text only labels. In general the disparity in number of entries is less extreme in the text only labels. <br>
<!-- TODO: find better phrasing -->
The presence of a large number of entries with the "something else" label suggests that the emotions used as labels are not sufficient to express the range of emotions contained in the dataset.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (13, 9))
for i, ax in enumerate(axs.reshape(-1)):
    ax.bar(freq_counts[cols[i]].keys(), freq_counts[cols[i]].values())
    ax.set_title(cols[i])
    bars = ax.bar(freq_counts[cols[i]].keys(), freq_counts[cols[i]].values())
    ax.bar_label(bars, freq_counts[cols[i]].values())
    labels = ax.get_xticklabels()
    for i, label in enumerate(labels):
        label.set_y(label.get_position()[1] - (i % 2) * 0.075)
fig.tight_layout(pad=3)
plt.show()

There are also 34 entries which have an empty main gold multimodal label (which means none of the emotions have a score of at least 2). For the text only label there are 11 empy entries. These will need to be handled in some way. A possible solution might be to take the labels considering scores of at least 1.

In [ ]:
df.loc[df["label_M_gold_main"].apply(len)==0].shape

Double check on score columns as well instead of list with labels. The number of samples where no emotion has a score greater than or equal to 2 is the same in both cases

In [ ]:
# mask checking for scores greater than or equal to 2
mask = df[[f"M_{x.capitalize()}" for x in emotions]].ge(2)

# check if on a row there is at least one emotion with score greater than or equal to 2 and print number of rows where it is true or false
mask.any(axis=1).value_counts()

Number of samples to remove considering each tweet-image pair as a separate sample

In [ ]:
sum = 0
for i, row in df[~mask.any(axis=1)].iterrows():
    sum += row["img_count"]
sum

In [ ]:
df.loc[df["label_T_gold_main"].apply(len)==0].shape

In [ ]:
# mask checking for scores greater than or equal to 2
mask = df[[f"T_{x.capitalize()}" for x in emotions]].ge(2)

# check if on a row there is at least one emotion with score greater than or equal to 2 and print number of rows where it is true or false
mask.any(axis=1).value_counts()

Number of samples to remove considering each tweet-image pair as a separate sample

In [ ]:
sum = 0
for i, row in df[~mask.any(axis=1)].iterrows():
    sum += row["img_count"]
sum

Most samples have multiple labels, even when considering a threshold of 2. Some emotions appear by themselves much less often than others. The ones that appear alone most often are joy, neutral and something else.

In [ ]:
cols = df.columns[df.columns.str.startswith("label_")].tolist()
freq_counts = {e: dict.fromkeys(emotions, 0) for e in cols}

for e in emotions:
    for column in cols:
        freq_counts[column][e] = df[(df[column].apply(lambda x: e in x)) & (df[column].apply(len)==1)].shape[0]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize = (13, 9))
for i, ax in enumerate(axs.reshape(-1)):
    ax.bar(freq_counts[cols[i]].keys(), freq_counts[cols[i]].values())
    ax.set_title(cols[i])
    bars = ax.bar(freq_counts[cols[i]].keys(), freq_counts[cols[i]].values())
    ax.bar_label(bars, freq_counts[cols[i]].values())
    labels = ax.get_xticklabels()
    for i, label in enumerate(labels):
        label.set_y(label.get_position()[1] - (i % 2) * 0.075)
fig.tight_layout(pad=3)
plt.show()

In [ ]:
cols

In [ ]:
for i, col in enumerate(cols[:2]):
    fig, ax= plt.subplots(figsize = (6,4))
    # ax.bar(freq_counts[col].keys(), freq_counts[col].values())
    # ax.set_title(cols[i])
    bars = ax.bar([e.capitalize() for e in emotions], freq_counts[col].values())
    ax.bar_label(bars, freq_counts[col].values())
    # labels = ax.get_xticklabels()
    # for i, label in enumerate(labels):
    #     label.set_y(label.get_position()[1] - (i % 2) * 0.055)
    ax.set_ylabel("Number of tweets")
    plt.xticks(rotation=45, ha='right')

# fig.tight_layout(pad=3)
plt.show()

Plotting distribution of number of votes for each emotion.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

options = df.columns[df.columns.str.startswith("M_")].values.tolist()
options = [x[2:] for x in options]

drop_down = widgets.Dropdown(options = options,
                             value = "Joy",
                             description = "Emotion",
                             disabled = False)

def dropdown_handler(emotion):
    
    fig, axes = plt.subplots(1, 2, figsize=(15,6))

    values = df[f"M_{emotion}"].value_counts()

    bars = axes[0].bar(values.index.values, values.values)
    axes[0].bar_label(bars, values.values)
    axes[0].set_title(f"Score distribution for {emotion} (Multimodal)")

    values = df[f"T_{emotion}"].value_counts()

    bars = axes[1].bar(values.index.values, values.values)
    axes[1].bar_label(bars, values.values)
    axes[1].set_title(f"Score distribution for {emotion} (Text only)")
    
    plt.show()
    plt.close()

w = interactive(dropdown_handler, emotion = drop_down)
display(w)

In [ ]:
df.M_Joy.max()

In [ ]:
cols = df.columns[df.columns.str.startswith("M_")]

count_dict = {emotion: None for emotion in emotions}

tmp = df[cols].astype(int)

for col, emotion in zip(cols, emotions):
    count_dict[emotion] = tmp[col].value_counts().sort_index()
for key, value in count_dict.items():
    for i in range(len(value)-2, -1, -1):
        value.iloc[i] += value.iloc[i+1]
tmp = pd.DataFrame.from_dict(count_dict).fillna(0).astype(int)
tmp = tmp.drop(0)
ax = tmp.plot(kind="bar", figsize=(10,4), rot=0)
ax.legend([e.capitalize() for e in emotions])
ax.set_xlabel("Threshold")
ax.set_ylabel("Number of tweets")
plt.show()

In [ ]:
tmp.columns = tmp.columns.str.capitalize()

In [ ]:
print(tmp.T.to_latex())

In [ ]:
cols = df.columns[df.columns.str.startswith("M_")]

rename_dict = {col: emotion for col, emotion in zip(cols, emotions)}

print(df[cols].astype(bool).sum(axis=0).rename(rename_dict).to_latex())


In [ ]:
cols = df.columns[df.columns.str.startswith("M_")]

rename_dict = {col: emotion for col, emotion in zip(cols, emotions)}

print((df[cols]>=2).sum(axis=0).rename(rename_dict).to_latex())

In [ ]:
df[df.columns[df.columns.str.startswith("M_") | df.columns.str.startswith("T_")].values.tolist()].mean()

Plot of the number of images for each emotion where the sample only has that emotion

In [ ]:
drop_down = widgets.Dropdown(options = emotions,
                             description = "Emotion",
                             value = "something else",
                             disabled = False)

def dropdown_handler(emotion):
    
    fig, axes = plt.subplots(1, 2, figsize=(15,6))

    values = df.loc[df["label_M_gold_main"].apply(lambda x : emotion in x and len(x)==1)]["img_count"].value_counts()

    bars = axes[0].bar(values.index.values, values.values)
    axes[0].bar_label(bars, values.values)
    axes[0].set_title(f"Number of images for {emotion} only samples (Multimodal)")

    values = df.loc[df["label_T_gold_main"].apply(lambda x : emotion in x and len(x)==1)]["img_count"].value_counts()

    bars = axes[1].bar(values.index.values, values.values)
    axes[1].bar_label(bars, values.values)
    axes[1].set_title(f"Number of images for {emotion} only samples (Text only)")

    
    plt.show()
    plt.close()


w = interactive(dropdown_handler, emotion = drop_down)
display(w)

Plot of the number of images for each emotion where the sample has that emotion as one of its labels

In [ ]:
drop_down = widgets.Dropdown(options = emotions,
                             description = "Emotion",
                             value = "something else",
                             disabled = False)

def dropdown_handler(emotion):
    
    fig, axes = plt.subplots(1, 2, figsize=(15,6))

    values = df.loc[df["label_M_gold_main"].apply(lambda x : emotion in x)]["img_count"].value_counts()

    bars = axes[0].bar(values.index.values, values.values)
    axes[0].bar_label(bars, values.values)
    axes[0].set_title(f"Number of images for {emotion} samples (Multimodal)")

    values = df.loc[df["label_T_gold_main"].apply(lambda x : emotion in x)]["img_count"].value_counts()

    bars = axes[1].bar(values.index.values, values.values)
    axes[1].bar_label(bars, values.values)
    axes[1].set_title(f"Number of images for {emotion} samples (Text only)")

    
    plt.show()
    plt.close()


w = interactive(dropdown_handler, emotion = drop_down)
display(w)

Number of samples to be dropped:
<ul>
    <li> 96 for multimodal labels (138 considering a sample for each image)</li>
    <li> 70 for txt only labels (95 considering a sample for each image)</li>
</ul>

In [ ]:
empty_label_samples = df.loc[df["label_M_gold_main"].apply(len)==0].shape[0]
print(empty_label_samples + freq_counts["label_M_gold_main"]["something else"])

In [ ]:
freq_counts["label_M_gold_main"]["something else"]

In [ ]:
label = "label_M_gold_main"
tot = 0
for i, row in df.iterrows():
    if len(row[label])==0 or ("something else" in row[label] and len(row[label])==1):
        tot += row["img_count"]
print(tot)

In [ ]:
empty_label_samples = df.loc[df["label_T_gold_main"].apply(len)==0].shape[0]
print(empty_label_samples + freq_counts["label_T_gold_main"]["something else"])

In [ ]:
label = "label_T_gold_main"
tot = 0
for i, row in df.iterrows():
    if len(row[label])==0 or ("something else" in row[label] and len(row[label])==1):
        tot += row["img_count"]
print(tot)

## Difference between silver and gold labels

Given that only a small part of the dataset has gold labels it is a good idea to check how much the gold and silver labels differ from one another so as to know the quality of the silver labels.<br>

There are two types of silver labels: "uni_label" and "multi_label". The first one contains only the highest scoring label, the second one is a list of all the emotions with a non-zero score. The score is the one given by the seeds, which are the words contained in the tweet which carry some kind of emotional meaning.

To compare the gold labels with the "uni_label" it is enough to check if the "uni_label" appears in the gold label as well. This holds for all types of gold labels.<br>

From the bar plot we can see that the silver label is quite inaccurate. Even considering the more lax gold labels only slightly more than half of the samples have a label that appears in the gold label. With the more restrictive gold labels this decreases to around a third of the samples.

In [ ]:
cols = df.columns[df.columns.str.startswith("label_")].tolist()
count_dict = {}
for column in cols:
    count = 0
    for i, row in df.iterrows():
        if row["uni_label"] not in row[column]:
            count += 1
    count_dict[column] = count

fig, ax = plt.subplots()

bars = ax.bar(count_dict.keys(), count_dict.values())
ax.bar_label(bars, count_dict.values())
labels = ax.get_xticklabels()
for i, label in enumerate(labels):
    label.set_y(label.get_position()[1] - (i % 2) * 0.075)
ax.set_title("Difference with uni label")
plt.show()
plt.close()

For the "multi_label" the comparison is slightly more involved, as it is a list instead of a single element. A rough way to do it is by checking if said list is disjoint with the gold label.<br>

Compared to the "uni_label" these silver labels have slightly better results, but this is partially because of the way the comparison is done, as even a single emotion being in both labels counts as a positive example. Thus, results are still quite bad in this case as well.

In [ ]:
for column in cols:
    count = 0
    for i, row in df.iterrows():
        if set(row["multi_label"]).isdisjoint(row[column]):
            count += 1
    count_dict[column] = count

fig, ax = plt.subplots()

bars = ax.bar(count_dict.keys(), count_dict.values())
ax.bar_label(bars, count_dict.values())
labels = ax.get_xticklabels()
for i, label in enumerate(labels):
    label.set_y(label.get_position()[1] - (i % 2) * 0.075)
ax.set_title("Difference with multi label")
plt.show()
plt.close()

Difference between labels by emotion

In [ ]:
cols = df.columns[df.columns.str.startswith("M_")].values.tolist()
cols = [x[2:].lower() for x in cols]
label_types = ["label_M_gold_main", "label_M_gold_multi"]

for type in label_types:

    counter_dict = {col:0 for col in cols}

    for i, row in df.iterrows():
        for label in row[type]:
            if label in row["uni_label"]:
                counter_dict[label] += 1


    fig, ax = plt.subplots()

    bars = ax.bar(counter_dict.keys(), counter_dict.values())
    ax.bar_label(bars, counter_dict.values())
    labels = ax.get_xticklabels()
    for i, label in enumerate(labels):
        label.set_y(label.get_position()[1] - (i % 2) * 0.075)
    ax.set_title(f"Difference with multi label by emotion for {type}")
    plt.show()
    plt.close()

In [ ]:
cols = df.columns[df.columns.str.startswith("M_")].values.tolist()
cols = [x[2:].lower() for x in cols]
label_types = ["label_M_gold_main", "label_M_gold_multi"]

for type in label_types:

    counter_dict = {col:0 for col in cols}

    for i, row in df.iterrows():
        for label in row[type]:
            if label in row["multi_label"]:
                counter_dict[label] += 1


    fig, ax = plt.subplots()

    bars = ax.bar(counter_dict.keys(), counter_dict.values())
    ax.bar_label(bars, counter_dict.values())
    labels = ax.get_xticklabels()
    for i, label in enumerate(labels):
        label.set_y(label.get_position()[1] - (i % 2) * 0.075)
    ax.set_title(f"Difference with multi label by emotion for {type}")
    plt.show()
    plt.close()

In [ ]:
cols = df.columns[df.columns.str.startswith("M_")].values.tolist()
cols = [x[2:].lower() for x in cols]
label_types = ["label_M_gold_main", "label_M_gold_multi"]

counter_dict = {type: {col:0 for col in cols} for type in label_types}

for type in label_types:
    for i, row in df.iterrows():
        for label in row[type]:
            if label in row["multi_label"]:
                counter_dict[type][label] += 1

pd.DataFrame(counter_dict).plot(kind="bar")

In [ ]:
count = 0
for i, row in df.iterrows():
    count += len(row["seeds"].keys())
count/(i+1)

## Difference between text only and multimodal labels

An interesting comparison is between the multimodal and text only gold labels, as it allows us to get an early look at how much the perception of emotions changes when one or more images are added to the context as opposed to classification done only on text. As before, one way to do this comparison is by checking if the sets are disjoint.<br>

Results differ quite a bit between the two types of gold label (depending on selection method, score higher than 1 or higher than 2). When considering a score higher than 2, thus a more selective gold label, 214 samples have completely disjoint gold labels, suggesting that images can, rather expectedly, make a difference to how the text is perceived by a reader.<br>

With the less restrictive gold labels there is more overlap, which is to be expected simply because of the labels containing more emotions overall and the comparison checking only if they share at least one element.<br>

One way to further analyze this topic might be checking how much the labels acrually overlap between each other instead of checking only if there is any overlap at all.

In [ ]:
count = 0
for i, row in df.iterrows():
    if set(row["label_M_gold_main"]).isdisjoint(row["label_T_gold_main"]):
        count += 1
print(count)

In [ ]:
count = 0
for i, row in df.iterrows():
    if set(row["label_M_gold_multi"]).isdisjoint(row["label_T_gold_multi"]):
        count += 1
print(count)

## Correlation between label scores

The correlation matrix is about what one might expect in this case.<br>

Correlation tends to be high when comparing the text only and multimodal scores of the same emotion. A notable exception to this is for the label "something else". The two labels have a correlation of 0.22, which, while positive, is lower than for the other labels. This might be caused by the fact that what is considered "something else" changes based on the presence or lack of images.<br>

The correlation between different emotions is also not particularly surprising. In general positive emotions correalate positively among themselves and negative emotions do so as well, albeit more strongly than positive ones. Positive and negative emotions have negative correlation with each other too. Neutral and "something else" are also inversely correlated with every other label.

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(figsize=(10, 7)) 
cols = df.columns[df.columns.str.startswith("M_")].tolist()
sns.heatmap(df[cols].corr(), annot = True, fmt = '.3f', ax=ax,
             square=True)
ax.set_xticklabels([e.capitalize() for e in emotions])
ax.set_yticklabels([e.capitalize() for e in emotions])
plt.xticks(rotation=45, ha='right') 
plt.show()
plt.close()

fig, ax = plt.subplots(figsize=(10, 7)) 
cols = df.columns[df.columns.str.startswith("M_")].tolist()
sns.heatmap((df[cols]>=2).corr(), annot = True, fmt = '.3f', ax=ax,
             square=True)
ax.set_xticklabels([e.capitalize() for e in emotions])
ax.set_yticklabels([e.capitalize() for e in emotions])
plt.xticks(rotation=45, ha='right') 
plt.show()
plt.close()

## Top images for each emotion

The top 10 images for each emotion are plotted here, together with the corresponding tweets.<br>

In [ ]:
import ipyplot

In [ ]:
tweets = []
images = []
labels = []
for e in emotions:
    label = f"M_{e.capitalize()}"
    for i, row in df.sort_values(label, ascending=False).head(10).iterrows():
        if e == "joy":
            print(row)
        tweets.append(row["tweet"])
        images.append(f"dataset/gold_images/twint_images3/{row['id']}_0.jpg")
        labels.append(label)

ipyplot.plot_class_tabs(images, labels, custom_texts=tweets, show_url=False, img_width=150)

In [ ]:
images = {e:[] for e in emotions}
tweets = {e:[] for e in emotions}
for e in emotions:
    label = f"M_{e.capitalize()}"
    for i, row in df.sort_values(label, ascending=False).head(5).iterrows():
        images[e].append(f"dataset/gold_images/twint_images3/{row['id']}_0.jpg")
        tweets[e].append(row["tweet"])
imgs = images["joy"]

In [ ]:
import regex as re
table = ""
build_path = lambda x: "images/dataset/top5/" + re.search("(?<=dataset/gold_images/twint_images3/)(.*).jpg", x).group(1) + ".pdf"
includegraphics = lambda x: f"\\centerincludegraphics[height=2cm]{{{x}}}"
count = 0
for e, imgs in images.items():
    table += "\\begin{subfigure}[b]{\\textwidth}\n\\centering\n"
    for image in imgs:
        table += includegraphics(build_path(image)) + "\n\\hfill\n"
    table += f"\\caption{{{e.capitalize()}}}\n\end{{subfigure}}\n\n"
print(table)

In [ ]:
import img2pdf
from PIL import Image
 
convert_imgs = False

if convert_imgs:
    # storing image path
    
    # storing pdf path
    for e, imgs in images.items():
        pdfs = ["./Tesi/images/dataset/top5/" + re.search("(?<=dataset/gold_images/twint_images3/)(.*).jpg", x).group(1) + ".pdf" for x in imgs]
        for img, pdf in zip(imgs, pdfs):
            # opening image
            image = Image.open(img)
            
            # converting into chunks using img2pdf
            pdf_bytes = img2pdf.convert(image.filename)
            
            # opening or creating pdf file
            file = open(pdf, "wb")
            
            # writing pdf files with chunks
            file.write(pdf_bytes)
            
            # closing image file
            image.close()
            
            # closing pdf file
            file.close()
        
    # output
    # print(pdf_path)

In [ ]:
import os
import shutil
if convert_imgs:
    src_files = imgs
    for file_name in src_files:
        # full_file_name = os.path.join(src, file_name)
        if os.path.isfile(file_name):
            shutil.copy(file_name, "C:/Users/Utente/Desktop/Multimodal-Sentiment-Analysis/Tesi/images/dataset/top5")

In [ ]:
tweets = []
images = []
labels = []
for e in emotions:
    label = f"T_{e.capitalize()}"
    for i, row in df.sort_values(label, ascending=False).head(10).iterrows():
        tweets.append(f"{i} " + row["tweet"])
        images.append(f"dataset/gold_images/twint_images3/{row['id']}_0.jpg")
        labels.append(label)

ipyplot.plot_class_tabs(images, labels, custom_texts=tweets, show_url=False, img_width=150)

## Wordclouds

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import re

In [ ]:
# function to plot wordclouds

# df is the dataset to use to plot the wordcloud
# multimodal is a flag to know which labels to use
# threshold is the minimum score to have in labels to be included in the wordcloud
def plot_wordcloud(df:pd.DataFrame, multimodal:bool, threshold:float = 0):
    fig, axs = plt.subplots(2, 5, figsize=(25, 10))

    if multimodal:
        label_type = "M"
    else:
        label_type = "T"

    for e, ax in zip(emotions, axs.flat):
        label = f"{label_type}_{e.capitalize()}"
        combined_text = ' '.join(df.loc[df[label] >= threshold]['tweet'])

        pattern = re.compile("&amp")
        combined_text = re.sub(pattern, "", combined_text)

        # cleaning the tweet text, code inspired from https://medium.com/codex/making-wordcloud-of-tweets-using-python-ca114b7a4ef4
        
        # remove emoji
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642" 
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  # dingbats
            u"\u3030"
            "]+", flags=re.UNICODE)
        combined_text = re.sub(emoji_pattern,'',combined_text)

        # remove urls
        url_pattern = re.compile(r'https?://\S+|www\.\S+?')
        combined_text = re.sub(url_pattern,'', combined_text)

        # remove @ mentions and hashes
        hash_pattern = re.compile("#")
        combined_text = re.sub(hash_pattern,"",combined_text)

        mention_pattern = re.compile("@[A-Za-z0–9_]+")
        combined_text = re.sub(mention_pattern,"",combined_text)
        
        # remove occurrences of &amp;
        and_pattern = re.compile("&amp;")
        combined_text = re.sub(and_pattern,"&",combined_text)

        wordcloud = (
            WordCloud(max_font_size=70, 
                    max_words=80,       
                    background_color="white",
                    min_word_length=3,
                    width=500,
                    height=500)
            .generate(combined_text)
        )
        ax.set_title(e.capitalize(), fontsize=18)
        ax.imshow(wordcloud)
        ax.axis("off")

    # if multimodal:
    #     plt.suptitle("Multimodal wordclouds", fontsize=30)
    # else:
    #     plt.suptitle("Text only wordclouds", fontsize=30)
    
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()
    plt.close()

In the wordcloud for tweets labeled with "fear" there are less words than other wordclouds. A possible explanation is that there are less tweets with that label. The same does not happen for "trust" though, despite a similar number of samples. "anticipation" also has fewer words than other labels, but only slightly.

In [ ]:
plot_wordcloud(df=df, multimodal=True, threshold=2)

Similar behaviour to multimodal, though "sadness" and "something else" also have sparse wordclouds. "fear" is less sparse compared to multimodal wordclouds.

In [ ]:
plot_wordcloud(df=df, multimodal=False, threshold=2)